In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import spacy
from spacy.language import Language
from spacy import displacy
import time
import glob
import re
import math
import statistics
import os
import json
import calendar
import holidays
from pathlib import Path
from datetime import date
from datetime import datetime
import pandas as pd
import numpy as np
import collections
import hashlib
from dateutil.parser import parse
import shutil
import ast
from io import StringIO
import requests

In [2]:
import hashlib
def getHash(sent):
    hash_object = hashlib.sha1(sent.upper().encode("UTF-8"))
    hex_dig = hash_object.hexdigest()
    return(hex_dig)

In [4]:
sym = "APPN"
entityFile = "../../Summary/entities/"+sym+"-ENTITIES.json"
print(entityFile)
#search = "Q\d+-[0-9][0-9][0-9][0-9]|ALL-[0-9][0-9][0-9][0-9]"
search = "Q\d+-[0-9][0-9][0-9][0-9]"
print(search)
cnt = 0
sentER = dict()
with open(entityFile, encoding="utf-8") as f:
    entity = json.load(f)
if(entity):
    for key in entity[sym].keys():
        if(re.search(search, key)):
            if("ALL" not in key):
                cnt = cnt + 1
            #if(cnt == 1):
            #    continue
            if(cnt > 5):
                break
            for ent in entity[sym][key]:
                if("PREV" not in ent and type(entity[sym][key][ent]) == dict and "RELATION" in entity[sym][key][ent] and len(entity[sym][key][ent]["RELATION"]) > 0):
                    for r in entity[sym][key][ent]["RELATION"]:
                        newent = ent.replace("-GUIDEFULL","")
                        newent = newent.replace("-GUIDE","")
                        newent = re.sub("-"+search, "", newent)
                        sent = r.split(":::")[2]
                        sent = sent.strip()
                        hexDig = getHash(sent)
                        relation = "{METRIC:"+newent+","
                        if hexDig not in sentER:
                            sentER[hexDig] = dict()
                        else:
                            if("RELATION" in sentER[hexDig] and relation in sentER[hexDig]["RELATION"][0]):
                                print("SAME RELATION " + relation + " ALREADY PRESENT FOR " + sent)
                                continue
                        if("RTEXT-MONEY" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                            relation = relation + "MONEY:"+entity[sym][key][ent]["RTEXT-MONEY"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE}"
                        elif("RTEXT-CD" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                            relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE}"
                        elif("RTEXT-PCT" in entity[sym][key][ent] and "RTEXT-RANGEM1" in entity[sym][key][ent]):
                            relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RANGEM1:"+entity[sym][key][ent]["RTEXT-RANGEM1"]+",RELATION:KVRANGE}"
                        elif("RTEXT-MONEY" in entity[sym][key][ent]):
                            relation = relation + "MONEY:"+entity[sym][key][ent]["RTEXT-MONEY"]+",RELATION:KV}"
                        elif("RTEXT-CD" in entity[sym][key][ent]):
                            relation = relation + "CD:"+entity[sym][key][ent]["RTEXT-CD"]+",RELATION:KV}"
                        elif("RTEXT-PCT" in entity[sym][key][ent]):
                            relation = relation + "PCT:"+entity[sym][key][ent]["RTEXT-PCT"]+",RELATION:KV}"
                        #print(key, sent+"\t"+relation, hexDig)
                        sentER[hexDig]["ORIGSENT"] = sent
                        sentER[hexDig]["KEY"] = key
                        if("RELATION" not in sentER[hexDig]):
                            sentER[hexDig]["RELATION"] = list()
                        sentER[hexDig]["RELATION"].append(relation)
print(sentER)

../../Summary/entities/APPN-ENTITIES.json
Q\d+-[0-9][0-9][0-9][0-9]
{'c437f9d89e3898b63db59ec5b036ec7191aae864': {'ORIGSENT': 'GAAP Net Loss Per Share is $(0.58) in second quarter 2023 @@@', 'KEY': 'Q2-2023', 'RELATION': ['{METRIC:GAAP-EPS,MONEY:-$0.58,RELATION:KV}']}, 'bcccbc5915db9fff595745a8c0d90b4d90379ce7': {'ORIGSENT': 'GAAP net loss per share is $(.58) for the second quarter of 2023.', 'KEY': 'Q2-2023', 'RELATION': ['{METRIC:GAAP-EPS,MONEY:-$0.58,RELATION:KV}']}, '610d85027497d0398976315ce0edfab7975a701a': {'ORIGSENT': 'GAAP Gross Profit is $90870 T in second quarter 2023 @@@', 'KEY': 'Q2-2023', 'RELATION': ['{METRIC:GAAP GROSS PROFIT,MONEY:$90.9MN,RELATION:KV}']}, '72f4930f4c3b5cbfb78c8ba4c2febf86983c1a4c': {'ORIGSENT': 'GAAP Gross Margin is 71.15% in second quarter 2023 @@@', 'KEY': 'Q2-2023', 'RELATION': ['{METRIC:GAAP GROSS MARGIN,PCT:71.15%,RELATION:KV}']}, '57bf919828be26218e1528a50fca02dbf1c6bfbe': {'ORIGSENT': 'Cloud subscription revenue is $74.4 million.', 'KEY': 'Q2-20

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [3]:
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Toss the pasta with the sauce, then add the meat and toss again.']


In [4]:
inputs = tokenizer("Get the Named entity from this sentence: Total revenue is expected to be between $115.0 million and $117.0 million, representing a year-over-year increase of 24% to 27%.", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Total revenue']


In [5]:
advanced_ner = """Microsoft Corporation is a company that makes computer software and video games. Bill Gates and Paul Allen founded the company in 1975 
[Company]: Microsoft, [Founded]: 1975, [Founders]: Bill Gates, Paul Allen 
 
Amazon.com, Inc., known as Amazon , is an American online business and cloud computing company. It was founded on July 5, 1994 by Jeff Bezos 
[Company]: Amazon, [Founded]: 1994, [Founders]: Jeff Bezos 
 
Apple Inc. is a multinational company that makes personal computers, mobile devices, and software. Apple was started in 1976 by Steve Jobs and Steve Wozniak."""
inputs = tokenizer(advanced_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['[Company]: Apple, [Founded]: 1976, [Founders]:']


In [6]:
report_ner = """"Revenue is $503MN up 77% year over year
[Metric]: Revenue, [Money]: $503MN, [PCT]: 77%, [CALENDAR]: year over year

Revenue is $603MN up 87% year over year
[Metric]: Revenue, [Money]: $603MN, [PCT]: 87%, [CALENDAR]: year over year

Revenue is $63MN down 12% year over year.
"""
inputs = tokenizer(report_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['"Revenue is $503MN up 77% year over year [Metric]:']
